In [1]:
pandas

NameError: name 'pandas' is not defined

In [2]:
pd

NameError: name 'pd' is not defined

In [1]:
import pandas as pd

In [2]:
from royaltyapp.income.helpers import StatementFactory, find_distinct_version_matching_errors, find_distinct_track_matching_errors, find_distinct_refund_matching_errors

In [3]:
from flask import Flask

In [4]:
import os

In [10]:
path = os.getcwd() + '/tests/api/test_income_returns/bandcamp.csv'

In [11]:
encoding_options = 'utf_16'
dtype_options = { 'catalog number': 'str', 'sku': 'str', 'upc': 'str' }

In [12]:
import numpy as np

In [13]:
df = pd.read_csv(path, encoding=encoding_options, dtype=dtype_options)
pd.set_option('display.max_columns', None)


In [14]:
df.loc[df['item type'] == 'reversal'].groupby(['bandcamp transaction id']).sum()

,item price,quantity,discount code,sub total,tax,shipping,transaction fee,item total,amount you received,assessed revenue share,collected revenue share,balance of revenue share,change to payout balance,payout balance,net amount,option,isrc,buyer name,buyer email,buyer phone,buyer note,ship to name,ship to street,ship to street 2,ship notes
bandcamp transaction id,,,,,,,,,,,,,,,,,,,,,,,,,
t212315173,49,-5.0,0.0,-83,0.0,-39.0,-5.55,-122,0.0,-11.7,-11.7,0.0,-146.75,136.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
total_price = df.loc[df['item type'] == 'reversal'].groupby(['bandcamp transaction id']).sum()[['item total']]

In [16]:
reversal = df.loc[df['item type'] == 'reversal'][['bandcamp transaction id', 'item total']]

In [17]:
sum = reversal.reset_index().merge(total_price, on='bandcamp transaction id')

In [18]:
change_to_payout = df.loc[df['item type'] == 'reversal'].groupby(['bandcamp transaction id']).sum()[['change to payout balance']]

In [19]:
joined = sum.merge(change_to_payout, on='bandcamp transaction id')

In [20]:
joined['new_net'] = joined['item total_x']/joined['item total_y']*joined['change to payout balance']

In [21]:
joined = joined.set_index('index')[['new_net']]

In [22]:
df = df.join(joined, lsuffix='_left', rsuffix='_right' )

In [23]:
types_to_change = ['refund', 'reversal']

In [26]:
df.loc[df['item type'].isin(types_to_change), 'net amount'] = df['new_net']

In [27]:
df.loc[df['item type'] == 'reversal']

,date,paid to,item type,item name,artist,currency,item price,quantity,discount code,sub total,tax,shipping,ship from country name,transaction fee,fee type,item total,amount you received,bandcamp transaction id,paypal transaction id,assessed revenue share,collected revenue share,balance of revenue share,change to payout balance,payout balance,net amount,package,option,item url,sku,catalog number,upc,isrc,buyer name,buyer email,buyer phone,buyer note,ship to name,ship to street,ship to street 2,ship to city,ship to state,ship to zip,ship to country,ship to country code,ship date,ship notes,country,country code,region/state,city,referrer,referrer url,new_net
3,1/20/20 7:32pm,Bandcamp,reversal,Amazing,Bonehead,USD,17,-2.0,NaN,-34,NaN,-8.0,United States,-5.55,credit card,-42,NaN,t212315173,NaN,-11.7,-11.7,0.0,-146.75,136.06,-50.520492,REVERSAL,NaN,REVERSAL,TEST-01lp,TEST-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Skopje,MKD,1020,Serbia,RS,NaN,NaN,Serbia,RS,MKD,Skopje,NaN,NaN,-50.520492
4,1/20/20 7:32pm,Bandcamp,reversal,Amazing,Bonehead,USD,17,-2.0,NaN,-34,NaN,-8.0,United States,NaN,NaN,-42,NaN,t212315173,NaN,NaN,NaN,NaN,NaN,NaN,-50.520492,REVERSAL,NaN,REVERSAL,TEST-01cass,TEST-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Skopje,MKD,1020,Serbia,RS,NaN,NaN,Serbia,RS,MKD,Skopje,Google search,www.google.com/,-50.520492
5,1/20/20 7:32pm,Bandcamp,reversal,Living the Dream,Various,USD,15,-1.0,NaN,-15,NaN,-23.0,United States,NaN,NaN,-38,NaN,t212315173,NaN,NaN,NaN,NaN,NaN,NaN,-45.709016,REVERSAL,NaN,REVERSAL,TEST-02lp,TEST-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Skopje,MKD,1020,Serbia,RS,NaN,NaN,Serbia,RS,MKD,Skopje,Google search,www.google.com/,-45.709016
